In [1]:
#default_exp datasets.genetic_file.Bgen

In [2]:
#hide
from nbdev.showdoc import *
import pandas as pd
from corradin_ovp_utils.catalog import test_data_catalog, conf_test_data_catalog
from corradin_ovp_utils.datasets.schemas import SingleFilePathSchema
import combinatorial_gwas
from fastcore.test import ExceptionExpected

In [3]:
#export
from corradin_ovp_utils.datasets.genetic_file import GeneticFileFormat, GeneticFileFormatInterfaceAPI, get_geno_one_snp, get_possible_geno_combinations
from typing import Any, Dict, List, Optional, Literal, Union, Protocol
from pydantic import BaseModel, PrivateAttr
import pandas as pd
from bgen_reader import open_bgen, read_bgen
from bgen_reader._bgen2 import open_bgen as open_bgen_file_type
import dask.dataframe as dd
from dask.delayed import Delayed
from fastcore.meta import delegates
import numpy as np
import logging
from collections import defaultdict

In [24]:
#export
def index_search(list_to_search, query_list):
    sorter = np.argsort(list_to_search)
    index = sorter[np.searchsorted(list_to_search, query_list, sorter=sorter)]
    return index

# class BgenFileObject(BaseModel):
#     variants: dd.DataFrame
#     samples: pd.Series
#     genotype: List[Delayed]
#     bgen_reader_obj: open_bgen
    
#     class Config:
#         arbitrary_types_allowed = True
    
#     def __repr__(self):
#         return str(self.__class__) + f" {self.samples.shape[0]} samples"
    
#     def get_variant_index(self,rsids=None):
#         variant_index = np.argwhere(np.isin(self.bgen_reader_obj.rsids, rsids)).reshape(-1,) if rsids is not None else None
#         return variant_index
    
#     def get_sample_index(self, sample_ids=None):
#         sample_index = np.argwhere(np.isin(self.samples.values, sample_ids)).reshape(-1,) if sample_ids is not None else None
#         return sample_index
    
#     def get_probs(self, sample_ids=None, rsids=None):
#         variant_index = self.get_variant_index(rsids)
#         sample_index = self.get_sample_index(sample_ids)
#         return self.bgen_reader_obj.read((sample_index, variant_index))
    
    
#     def get_geno_each_sample(self, probs, prob_to_geno_func:Literal["max", "stringent"]= "stringent", high_lim=0.9, low_lim=0.3, NA_val=np.nan):
#         if prob_to_geno_func == "max":
#             geno = np.nanargmax(probs, axis=2).astype(float)
            
#         elif prob_to_geno_func == "stringent":
#             geno = np.apply_along_axis(get_geno_one_snp, axis=2, arr=probs, high_lim=high_lim, low_lim=low_lim, NA_val=NA_val)
        
#         return geno
            
        
#     def get_allele_ids(self, rsids = None, variant_index = None):
#         if variant_index is None:
#             variant_index = self.get_variant_index(rsids)
#         df = pd.DataFrame([allele_str.split(",") for allele_str in self.bgen_reader_obj.allele_ids[variant_index]], columns = ["allele_1", "allele_2"])
        
#         if rsids is not None:
#             df.index = rsids
#         return df
    
#     def get_variant_combinations(self, rsids = None, variant_index = None):
#         if variant_index is None:
#             variant_index = np.argwhere(np.isin(self.bgen_reader_obj.rsids, rsids)).reshape(-1,) if rsids is not None else None
#         geno_df = self.get_allele_ids(rsids, variant_index)
#         geno_df = get_possible_geno_combinations(geno_df, "allele_1", "allele_2")
#         return geno_df
            
#     #def get_genotypes_df(self, rsid_list: List = None, **kwargs):


class BgenFileObject():
#     variants: dd.DataFrame
#     samples: pd.Series
#     genotype: List[Delayed]

    bgen_reader_obj: open_bgen
    
    def __init__(self, bgen_reader_obj):
        self.bgen_reader_obj = bgen_reader_obj
        self.samples = np.vstack(np.char.split(self.bgen_reader_obj.samples, sep = " "))[:,0]
    
    #to do: use fastcore "delegate" function
    @property
    def ids(self):
        return self.bgen_reader_obj.ids
    
    @property
    def rsids(self):
        return self.bgen_reader_obj.rsids
    
    def __repr__(self):
        return str(self.__class__) + f" {self.samples.shape[0]} samples"
    
    def get_variant_index(self,ids=None, id_type = "rsid"):
        if id_type == "id":
            variant_index = index_search(self.ids, ids) if ids is not None else None
        elif id_type == "rsid":
            variant_index = index_search(self.rsids, ids) if ids is not None else None
        return variant_index
    
    def get_sample_index(self, sample_ids=None):
        sample_index = index_search(self.samples, sample_ids) if sample_ids is not None else None
        return sample_index
    
    def get_probs(self, samples_index=None, variants_index = None):#get_probs(self, sample_ids=None, variant_ids=None):
        # variant_index = self.get_variant_index(variant_ids)
        # sample_index = self.get_sample_index(sample_ids)
        
        return self.bgen_reader_obj.read((samples_index, variants_index))
    
    
    def get_all_geno_df(self, variant_ids, variants_index):
        all_geno_df = pd.DataFrame({"id_col": variant_ids, "raw_alleles":self.bgen_reader_obj.allele_ids[variants_index]}).set_index("id_col")
        all_geno_df[["alleleA", "alleleB"]] = all_geno_df["raw_alleles"].str.split(",", expand = True)
        all_geno_df = get_possible_geno_combinations(all_geno_df, allele_1_col="alleleA", allele_2_col= "alleleB")
        return all_geno_df

    def get_geno_each_sample(self,
                             sample_ids=None,
                             variant_ids=None,
                             variant_id_type="rsid",
                             prob_to_geno_func:Union["max", "stringent"]= "stringent",
                             high_lim=0.9,
                             low_lim=0.3,
                             NA_val=np.nan,
                             geno_format: Literal["raw", "ohe", "str"]= "raw",
                             **kwargs):
        num_variants_to_search = len(variant_ids)
        variants_index = self.get_variant_index(ids = variant_ids, id_type= variant_id_type)
        
        if variant_id_type not in ["id", "rsid", "ids", "rsids"]:
            raise ValueError
        if variant_id_type in ["id", "rsid"]:
            variant_id_type += "s"
            
        #confirming that we have the correct variants by reindexing
        list_of_variants_in_file_specific_id_type = getattr(self.bgen_reader_obj, variant_id_type)
        confirmed_correct_variants_bool_mask = (list_of_variants_in_file_specific_id_type[variants_index]) == np.array(variant_ids)
        
        
        confirmed_correct_variants_index = variants_index[confirmed_correct_variants_bool_mask]
        found_variants = np.array(variant_ids)[confirmed_correct_variants_bool_mask]
        #assert all(self.bgen_reader_obj.rsids[variants_index] == variant_ids)
        not_found_variants = np.array(variant_ids)[~confirmed_correct_variants_bool_mask]
        
        logging.warning(f"\nFound variants: {found_variants.shape[0]}/{num_variants_to_search}\n Not found: {not_found_variants.shape[0]}/{num_variants_to_search}.\n Percent found {round((found_variants.shape[0]/num_variants_to_search) *100)}%")
        
        samples_index = self.get_sample_index(sample_ids = sample_ids)
        probs = self.get_probs(samples_index=samples_index, variants_index = confirmed_correct_variants_index)
        
        if prob_to_geno_func == "max":
            geno = np.nanargmax(probs, axis=2).astype(float)
   
        elif prob_to_geno_func == "stringent":
            geno = np.apply_along_axis(get_geno_one_snp, axis=2, arr=probs, high_lim=high_lim, low_lim=low_lim, NA_val=NA_val)
        
        

        geno_no_nan = np.nan_to_num(geno, nan=3).astype(int)
        all_geno_df = self.get_all_geno_df(variant_ids = found_variants, variants_index = confirmed_correct_variants_index)
        
        if geno_format == "ohe":
            geno_final = np.identity(4)[geno_no_nan]
        elif geno_format == "str":
            geno_final = self.process_raw_geno_data(raw_genos_matrix = geno, all_geno_df = all_geno_df, variant_ids = found_variants)
        else:
            geno_final = geno
        return geno_final, all_geno_df, found_variants, not_found_variants
    

    def process_raw_geno_data(self, raw_genos_matrix, all_geno_df, variant_ids):
        
        genos_str = np.select([raw_genos_matrix == 0,
                               raw_genos_matrix ==1,
                               raw_genos_matrix == 2],
                              choicelist=[all_geno_df["AA"], all_geno_df["AB"], all_geno_df["BB"]],
                              default = np.nan)
        
        all_samples_df = pd.DataFrame(genos_str, columns = variant_ids, index = self.samples)
        all_samples_df.index.name = "sample_id"
        return all_samples_df
        
    def get_allele_ids(self, rsids = None, variant_index = None):
        if variant_index is None:
            variant_index = self.get_variant_index(rsids)
        df = pd.DataFrame([allele_str.split(",") for allele_str in self.bgen_reader_obj.allele_ids[variant_index]], columns = ["allele_1", "allele_2"])
        
        if rsids is not None:
            df.index = rsids
        return df
    
    def get_variant_combinations(self, rsids = None, variant_index = None):
        if variant_index is None:
            variant_index = self.get_variant_index(rsids).reshape(-1,) if rsids is not None else None
        geno_df = self.get_allele_ids(rsids, variant_index)
        geno_df = get_possible_geno_combinations(geno_df, "allele_1", "allele_2")
        return geno_df
    
    
    
class BgenFileFormat(GeneticFileFormat, GeneticFileFormatInterfaceAPI):
    _all_chrom_dict: Dict[int, open_bgen_file_type] = PrivateAttr()
    # = PrivateAttr()
        
    class Config:
        arbitrary_types_allowed = True
        
    def __init__(self, **data):
        super().__init__(**data)
        self._all_chrom_dict = None #{chrom_number : self.load_one_chrom(chrom_number) for chrom_number in range(1,23)}
        self.sample_ids = None #self._all_chrom_dict[1].samples
        
    @property
    def _samples_int(self):
        return self.sample_ids.astype(int)
    
    def get_geno_matrix_specific_chrom(self, *, chrom, chrom_specific_variant_ids, sample_id_subset, variant_id_type):
        logging.warning(f"Loading {len(chrom_specific_variant_ids)} SNPs with id type '{variant_id_type}' for chromosome {chrom}, {len(sample_id_subset)} people")
        all_samples_df_specific_chrom, all_geno_df_specific_chrom, found_variants, not_found_variants = self._all_chrom_dict[chrom].get_geno_each_sample(prob_to_geno_func = "stringent",
                                                                                         sample_ids=sample_id_subset,
                                                                                         variant_ids=chrom_specific_variant_ids,
                                                                                         variant_id_type= variant_id_type,
                                                                                         geno_format = "str")
        return  all_samples_df_specific_chrom, all_geno_df_specific_chrom, found_variants, not_found_variants
    
    #TODO: accomodate different types of SNP id
    def get_geno_each_sample(self, rsid_dict:Dict[int, List[str]], id_col_list: List[str] = [], batch_size=1_000, excluded_sample_ids: List[str] =[]): 
        if self._all_chrom_dict is None or self.sample_ids is None:
            chromosomes_needed = list(rsid_dict.keys())
            logging.warning(f"Initialize dictionary of genetic files for {len(chromosomes_needed)} chromosomes, this might take a while")
            self._all_chrom_dict = {chrom_number : self.load_one_chrom(chrom_number) for chrom_number in rsid_dict.keys()}
            self.sample_ids = self._all_chrom_dict[chromosomes_needed[0]].samples
        all_samples_df_list = []
        all_geno_df_list = []
        found_variants_dict= defaultdict(list)
        not_found_variants_dict=defaultdict(list)
        
        sample_id_subset = list(set(self.sample_ids) - set(excluded_sample_ids))
        
        #print(sample_id_subset)
        for chrom, chrom_specific_variant_ids in rsid_dict.items():
            rsids_list = list(set([variant_id for variant_id in chrom_specific_variant_ids if variant_id.startswith("rs")]))
            if rsids_list:
                all_samples_df_specific_chrom, all_geno_df_specific_chrom, found_variants_specific_chrom, not_found_variants_specific_chrom = self.get_geno_matrix_specific_chrom(chrom = chrom,
                                                                                                                chrom_specific_variant_ids= rsids_list,
                                                                                                                sample_id_subset= sample_id_subset,
                                                                                                                variant_id_type = "rsid")

                all_samples_df_list.append(all_samples_df_specific_chrom)
                all_geno_df_list.append(all_geno_df_specific_chrom)
                found_variants_dict[chrom].append(found_variants_specific_chrom)
                not_found_variants_dict[chrom].append(not_found_variants_specific_chrom)
            
            ids_list = [id_str for id_str in (set(chrom_specific_variant_ids) - set(rsids_list)) if self.check_if_id(id_str)]
            if ids_list:
                all_samples_df_specific_chrom, all_geno_df_specific_chrom = self.get_geno_matrix_specific_chrom(chrom = chrom,
                                                                                                                chrom_specific_variant_ids= ids_list,
                                                                                                                sample_id_subset= sample_id_subset,
                                                                                                                variant_id_type = "id")

                all_samples_df_list.append(all_samples_df_specific_chrom)
                all_geno_df_list.append(all_geno_df_specific_chrom)
                found_variants_dict[chrom].append(found_variants_specific_chrom)
                not_found_variants_dict[chrom].append(not_found_variants_specific_chrom)
        
        not_found_variants_sum_dict = {chrom: len(variant_list) for chrom, variant_list in not_found_variants_dict.items()}
        found_variants_sum_dict = {chrom: len(variant_list) for chrom, variant_list in found_variants_dict.items()}
        
        total_variants_found = sum([sum_each_chrom for sum_each_chrom in found_variants_sum_dict.values()])
        total_variants_not_found = sum([sum_each_chrom for sum_each_chrom in not_found_variants_sum_dict.values()])
        logging.warning(f"Number of found variants per chromosome: {found_variants_sum_dict}")
        logging.warning(f"Number of not found variants per chromosome: {not_found_variants_sum_dict}")
        
        return pd.concat(all_samples_df_list), pd.concat(all_geno_df_list)
        
        
    def check_if_id(self, variant_string):
        chrom, rest = variant_string.split(":")
        if 1 <= chrom <= 22 and len(rest.split("_")) == 3:
            return True
        else:
            return False
    
    @delegates(read_bgen)
    def load_one_chrom(self, chrom=None, **kwargs):
        file_path = self.get_resolved_file_path(chrom=chrom)
        print(f"Loading chromosome {chrom}")
        open_bgen_obj = open_bgen(file_path, samples_filepath= self.sample_file, allow_complex = False)
        return BgenFileObject(bgen_reader_obj = open_bgen_obj)


In [27]:
test_file_path = SingleFilePathSchema(folder = "/lab/corradin_biobank/Raw_UKB_downloads/BGEN/",
                     full_file_name = "ukb_imp_chr{chrom_num}_v3.bgen", split_by_chromosome= True)

In [28]:
test_bgen_file_class_instance = BgenFileFormat( file_path = test_file_path,
                                               sample_file = "/lab/corradin_biobank/Raw_UKB_downloads/sample_files/ukb45624_imp_chr9_v3_s487320.sample")
test_bgen_file_chrom_22 = test_bgen_file_class_instance.load_one_chrom(22)

Loading chromosome 22


In [7]:
test_bgen_file_chrom_22.get_geno_each_sample(variant_id_type="id", variant_ids=['22:16050075_A_G', '22:16050213_C_T', '22:16212580_G_A'] )

Found variants: 3/3
 Not found: 0/3.
 Percent found 100%


reading -- time=0:00:00.00, thread 1 of 3, part 1 of 1


(array([[ 0.,  0., nan],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0., nan]]),                 alleleA alleleB  AA  AB  BB
 id_col                                     
 22:16050075_A_G       A       G  AA  AG  GG
 22:16050213_C_T       C       T  CC  CT  TT
 22:16212580_G_A       G       A  GG  AG  AA, array(['22:16050075_A_G', '22:16050213_C_T', '22:16212580_G_A'],
       dtype='<U15'), array([], dtype='<U15'))

In [13]:
test_variants_id_type = ['22:16050075_A_G', '22:16050213_C_T', '22:16212580_G_A']
test_variant_index = test_bgen_file_chrom_22.get_variant_index(ids = test_variants_id_type, id_type= "id")
test_variants_backcheck_id = test_bgen_file_chrom_22.ids[test_variant_index]
assert np.array_equal(test_variant_index, np.array([0, 2, 2000]))
assert all(test_variants_backcheck_id == test_variants_id_type)

In [14]:
test_variants_rsid_type = ["rs77948203", "rs9610458", "rs134490", "rs5756405"]
test_variant_index = test_bgen_file_chrom_22.get_variant_index(test_variants_rsid_type)
assert all(test_bgen_file_chrom_22.rsids[test_variant_index] == test_variants_rsid_type)

In [29]:
test_variants_mixed = test_variants_id_type + test_variants_rsid_type
test_variants_mixed_genos = test_bgen_file_class_instance.get_geno_each_sample(rsid_dict= {22: test_variants_mixed})


Loading chromosome 22


Found variants: 4/4
 Not found: 0/4.
 Percent found 100%


reading -- time=0:00:00.00, thread 1 of 4, part 1 of 1


TypeError: '<=' not supported between instances of 'int' and 'str'

In [17]:
test_variants_big = \
list(set(["rs10853845",
"rs2253675",
"rs2331964",
"rs10853845",
"rs10853845",
"rs10853845",
"rs134490",
"rs134490",
"rs4560284",
"rs134490",
"rs5756405",
"rs9610458",
"rs134490",
"rs5756405",
"rs9610458",
"rs201338837",
"rs201338837",
"rs201338837",
"rs229502",
"rs183032559",
"rs79963419",
"rs145985218",
"rs202193982",
"rs201338837",
"rs1004237",
"rs1003500",
"rs1014626",
"rs5762201",
"rs4821519",
"rs77948203",]))
print(test_variants_big)
print(len(test_variants_big))

['rs1014626', 'rs10853845', 'rs4560284', 'rs79963419', 'rs2331964', 'rs183032559', 'rs134490', 'rs202193982', 'rs77948203', 'rs5762201', 'rs201338837', 'rs4821519', 'rs2253675', 'rs5756405', 'rs9610458', 'rs1003500', 'rs145985218', 'rs229502', 'rs1004237']
19


In [18]:
test_variants_index_big = test_bgen_file_chrom_22.get_variant_index(test_variants_big)
test_all_rsids_single_chrom = test_bgen_file_chrom_22.rsids
test_confirmed_variants_bool_mask = (test_all_rsids_single_chrom[test_variants_index_big]) == np.array(test_variants_big)
test_confirmed_variants = np.array(test_variants_big)[test_confirmed_variants_bool_mask]
print(test_confirmed_variants)
print(len(test_confirmed_variants))

['rs1014626' 'rs79963419' 'rs183032559' 'rs134490' 'rs202193982'
 'rs77948203' 'rs5762201' 'rs4821519' 'rs5756405' 'rs9610458' 'rs1003500'
 'rs229502' 'rs1004237']
13


In [19]:

%timeit test_genos = test_bgen_file_class_instance.get_geno_each_sample(rsid_dict = {22: [test_variants[0]]})

Loading chromosome 22


Found variants: 1/1
 Not found: 0/1.
 Percent found 100%


reading -- time=0:00:00.00, thread 1 of 1, part 1 of 1


Found variants: 1/1
 Not found: 0/1.
 Percent found 100%


reading -- time=0:00:00.00, thread 1 of 1, part 1 of 1


...

reading -- time=0:00:00.00, thread 1 of 1, part 1 of 1


Found variants: 1/1
 Not found: 0/1.
 Percent found 100%


reading -- time=0:00:00.00, thread 1 of 1, part 1 of 1


Found variants: 1/1
 Not found: 0/1.
 Percent found 100%


reading -- time=0:00:00.00, thread 1 of 1, part 1 of 1
21.6 s ± 182 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [226]:
test_genos = test_bgen_file_class_instance.get_geno_each_sample(rsid_dict = {22: test_variants_big})

Loading chromosome 22


Found variants: 13/19
 Not found: 6/19.
 Percent found 68%


reading -- time=0:00:00.00, thread 1 of 13, part 1 of 1


In [227]:
test_genos#[22]#.shape

(          rs1004237 rs134490 rs1014626 rs77948203 rs4821519 rs202193982  \
 sample_id                                                                 
 5542886          CC       TT        TT         GG        GG          GG   
 5137974          CC       TT        TT         GG        GG          GG   
 3758348          CC       TT        TT         GG        GG          GG   
 1391800          CC       CT        TT         GG        GG          GG   
 3165331          CC       TT        TT         GG        GG          GG   
 ...             ...      ...       ...        ...       ...         ...   
 5512806          CC       CT        TT         AA        GG          GG   
 5548469          CC       TT        TT         GG        GG          GG   
 2956972          CC       TT        TT         GG        GG          GG   
 5229561          CC       TT        TT         AG        GG          GG   
 3665101          CC       TT        TT         GG        GG          GG   
 
          

In [76]:
all_geno_df = pd.DataFrame({"id_col": test_variants , "raw_alleles":test.bgen_reader_obj.allele_ids[test_variant_index]}).set_index("id_col")
all_geno_df[["alleleA", "alleleB"]] = all_geno_df["raw_alleles"].str.split(",", expand = True)
all_geno_df = get_possible_geno_combinations(all_geno_df, allele_1_col="alleleA", allele_2_col= "alleleB")
all_geno_df

,alleleA,alleleB,AA,AB,BB
id_col,,,,,
rs77948203,G,A,GG,AG,AA
rs9610458,C,T,CC,CT,TT
rs134490,C,T,CC,CT,TT
rs5756405,A,G,AA,AG,GG


In [79]:
test_genos_str = np.select([test_genos[22] == 0, test_genos[22] ==1, test_genos[22] == 2], choicelist=[all_geno_df["AA"], all_geno_df["AB"], all_geno_df["BB"]], default = np.nan)
test_genos_str

array([['GG', 'TT', 'TT', 'AA'],
       ['GG', 'CC', 'TT', 'AG'],
       ['GG', 'CT', 'TT', 'GG'],
       ...,
       ['GG', 'CT', 'TT', 'GG'],
       ['AG', 'CT', 'TT', 'GG'],
       ['GG', 'TT', 'TT', 'AG']], dtype=object)

In [85]:
all_samples_df = pd.DataFrame(test_genos_str, columns = test_variants, index = test.samples)
all_samples_df

,rs77948203,rs9610458,rs134490,rs5756405
5542886,GG,TT,TT,AA
5137974,GG,CC,TT,AG
3758348,GG,CT,TT,GG
1391800,GG,TT,CT,AG
3165331,GG,CT,TT,AA
...,...,...,...,...
5512806,AA,CT,CT,AA
5548469,GG,CC,TT,AG
2956972,GG,CT,TT,GG
5229561,AG,CT,TT,GG


In [81]:
test_genos_str.shape

(487409, 4)

---

---

In [38]:
test = test_data_catalog.load("genetic_file_bgen").files.single_file
test.get_resolved_file_path(2)

Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr2_v3.bgen')

In [41]:
test_bgen_file_format = test_data_catalog.load("genetic_file_bgen").files.single_file
test_bgen_file_obj = test_bgen_file_format.load_one_chrom(chrom=22)

Loading chromosome 22


In [43]:
test_bgen_file_obj.samples

array(['5542886', '5137974', '3758348', ..., '2956972', '5229561',
       '3665101'], dtype='<U7')

In [44]:
vars(test_bgen_file_obj)

{'bgen_reader_obj': <bgen_reader._bgen2.open_bgen at 0x14f7c2053970>,
 'samples': array(['5542886', '5137974', '3758348', ..., '2956972', '5229561',
        '3665101'], dtype='<U7')}

In [45]:
#redo this to rapidly change and reload the BgenFileObject code inside same file
test_bgen_file_obj = BgenFileObject(**{key: val for key, val in vars(test_bgen_file_obj).items() if key not in ["samples"]})

In [47]:
test_bgen_file_obj.get_variant_index(ids = ['22:16050075_A_G', '22:16050213_C_T', '22:16212580_G_A'], id_type= "id")

array([   0,    2, 2000])

In [9]:
variant_index = np.argwhere(np.isin(test_bgen_file_obj.bgen_reader_obj.rsids, ['9:10163_CT_C', '9:10273_AAC_A', '9:141146683_C_G', '9:141148854_G_A'])).reshape(-1,)
variant_index

array([      0,       1, 4066772, 4066773])

In [10]:
sample_index = np.argwhere(np.isin(test_bgen_file_obj.samples.values, ['5542886', '5137974', '3758348'])).reshape(-1,)
sample_index

array([0, 1, 2])

In [40]:
test_probs = test_bgen_file_obj.get_probs(sample_ids = ['5542886', '5137974', '3758348'], 
                                          rsids = ['9:10163_CT_C', '9:10273_AAC_A', '9:141146683_C_G', '9:141148854_G_A'])
test_probs

reading -- time=0:00:00.00, thread 1 of 4, part 1 of 1


array([[[1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [0.97647059, 0.02352941, 0.        ],
        [1.        , 0.        , 0.        ]],

       [[1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [0.98039216, 0.01960784, 0.        ],
        [1.        , 0.        , 0.        ]],

       [[1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [0.30588235, 0.69411765, 0.        ],
        [1.        , 0.        , 0.        ]]])

In [29]:
test_probs = test_bgen_file_obj.get_probs(sample_ids =['5542886'] , rsids = test_bgen_file_obj.bgen_reader_obj.rsids[:1_000_000])

reading -- time=0:15:52.81, thread 1 of 48, part 20,834 of 20,834


In [41]:
test_probs.nbytes

288

In [47]:
test_geno = test_bgen_file_obj.get_geno_each_sample(test_probs)
test_geno

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0., nan,  0.]])

In [197]:
test_geno_df = test_bgen_file_obj.get_allele_ids(rsids=['9:10163_CT_C', '9:10273_AAC_A', '9:141146683_C_G', '9:141148854_G_A'])
test_geno_df

,allele_1,allele_2
9:10163_CT_C,CT,C
9:10273_AAC_A,AAC,A
9:141146683_C_G,C,G
9:141148854_G_A,G,A


In [198]:
test_bgen_file_obj.get_variant_combinations(rsids=['9:10163_CT_C', '9:10273_AAC_A', '9:141146683_C_G', '9:141148854_G_A'])

,allele_1,allele_2,homo_1,het,homo_2
9:10163_CT_C,CT,C,CT/CT,C/CT,C/C
9:10273_AAC_A,AAC,A,AAC/AAC,A/AAC,A/A
9:141146683_C_G,C,G,C/C,C/G,G/G
9:141148854_G_A,G,A,G/G,A/G,A/A


In [ ]:
test_bgen_file_obj

In [96]:
test_bgen_file_obj.variants.query("nalleles !=2").head(npartitions = -1)

/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/.venv/lib/python3.8/site-packages/dask/dataframe/core.py:6558: UserWarning: Insufficient elements for `head`. 5 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,id,rsid,chrom,pos,nalleles,allele_ids,vaddr


In [124]:
test_bgen_file_obj.bgen_reader_obj.rsids

memmap(['9:10163_CT_C', '9:10273_AAC_A', 'rs535827433', ...,
        '9:141146445_T_TA', '9:141146683_C_G', '9:141148854_G_A'],
       dtype='<U151')

In [73]:
test_bgen_file_obj.get_probs(['5542886'], ['9:10163_CT_C', '9:10273_AAC_A', '9:141146683_C_G', '9:141148854_G_A']).shape

reading -- time=0:00:00.00, thread 1 of 4, part 1 of 1


(1, 4, 3)

In [123]:
test_arr = test_bgen_file_obj.get_probs(['5542886', '5137974', "1391800"], ['9:10163_CT_C', '9:10273_AAC_A', '9:141146683_C_G', '9:141148854_G_A'])
test_arr

reading -- time=0:00:00.00, thread 1 of 4, part 1 of 1


array([[[1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [0.97647059, 0.02352941, 0.        ],
        [1.        , 0.        , 0.        ]],

       [[1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [0.98039216, 0.01960784, 0.        ],
        [1.        , 0.        , 0.        ]],

       [[0.99215686, 0.00784314, 0.        ],
        [1.        , 0.        , 0.        ],
        [0.89411765, 0.10588235, 0.        ],
        [1.        , 0.        , 0.        ]],

       [[1.        , 0.        , 0.        ],
        [1.        , 0.        , 0.        ],
        [0.85098039, 0.14901961, 0.        ],
        [1.        , 0.        , 0.        ]]])

In [125]:
test_bgen_file_obj.bgen_reader_obj.allele_ids

memmap(['CT,C', 'AAC,A', 'T,C', ..., 'T,TA', 'C,G', 'G,A'], dtype='<U414')

In [126]:
test_bgen_file_obj.bgen_reader_obj.allele_ids[variant_index]

array(['CT,C', 'AAC,A', 'C,G', 'G,A'], dtype='<U414')

In [136]:
pd.DataFrame([allele_str.split(",") for allele_str in test_bgen_file_obj.bgen_reader_obj.allele_ids[variant_index]], columns = ["allele_1", "allele_2"])

,allele_1,allele_2
0,CT,C
1,AAC,A
2,C,G
3,G,A


In [93]:
np.apply_along_axis(lambda x: print(x), 2, test_arr)

[1. 0. 0.]
[1. 0. 0.]
[0.97647059 0.02352941 0.        ]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0.98039216 0.01960784 0.        ]
[1. 0. 0.]


array([[None, None, None, None],
       [None, None, None, None]], dtype=object)

In [116]:
np.argmax(test_arr, axis=2).shape

(2, 4)

In [121]:
np.argmax(test_arr, axis=2)

array([[0, 0, 0, 0],
       [0, 0, 0, 0]])

In [120]:
np.identity(3)[np.argmax(test_arr, axis=2)]

array([[[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]])

In [91]:
np.argmax(test_arr, axis=2)

array([[0, 0, 0, 0],
       [0, 0, 0, 0]])

In [ ]:
np.apply_along_axis(np.argmax)

In [ ]:
np.apply_over_axes()

In [86]:
test_bgen_file_obj.get_probs(['5542886', '5137974'], ['9:10163_CT_C', '9:10273_AAC_A', '9:141146683_C_G', '9:141148854_G_A'])[:,:,0]

reading -- time=0:00:00.00, thread 1 of 4, part 1 of 1


array([[1.        , 1.        , 0.97647059, 1.        ],
       [1.        , 1.        , 0.98039216, 1.        ]])

In [ ]:
np.apply()

In [82]:
test_bgen_file_obj.samples

0         5542886
1         5137974
2         3758348
3         1391800
4         3165331
           ...   
487404    5512806
487405    5548469
487406    2956972
487407    5229561
487408    3665101
Name: id, Length: 487409, dtype: object

In [57]:
test_bgen_file_obj.bgen_reader_obj.allele_ids

memmap(['CT,C', 'AAC,A', 'T,C', ..., 'T,TA', 'C,G', 'G,A'], dtype='<U414')

In [65]:
test_bgen_file_obj.bgen_reader_obj.read((sample_index, variant_index))

reading -- time=0:00:00.00, thread 1 of 4, part 1 of 1


(3, 4, 3)

In [36]:
test_bgen_file_obj.bgen_reader_obj.read(variant_index).shape

reading -- time=0:00:00.00, thread 1 of 4, part 1 of 1


(487409, 4, 3)

In [21]:
test_bgen_file_obj.variants.shape[0].compute()

4066774

In [11]:
conf_test_data_catalog["genetic_file_bgen"]

{'type': 'corradin_ovp_utils.datasets.OVPDataset.OVPDataset',
 'file_format': 'genetic_file.Bgen.BgenFileFormat',
 'file_type': 'OVPDataset.SingleFilePathSchema',
 'file_path': {'folder': '/lab/corradin_biobank/Raw_UKB_downloads/BGEN/',
  'split_by_chromosome': True,
  'full_file_name': 'ukb_imp_chr{chrom_num}_v3.bgen'},
 'load_args': {'sample_file': '/lab/corradin_biobank/Raw_UKB_downloads/sample_files/ukb45624_imp_chr9_v3_s487320.sample'}}

In [15]:
read_bgen("/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr9_v3.bgen", samples_filepath="/lab/corradin_biobank/Raw_UKB_downloads/sample_files/ukb45624_imp_chr9_v3_s487320.sample")

Sample IDs are read from /lab/corradin_biobank/Raw_UKB_downloads/sample_files/ukb45624_imp_chr9_v3_s487320.sample.


Mapping genotypes:   1%|          | 27635/4066774 [00:02<06:12, 10830.68it/s]

KeyboardInterrupt: 

Mapping genotypes:   1%|          | 27635/4066774 [00:21<06:12, 10830.68it/s]

In [24]:
test_bgen_obj = open_bgen("/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr9_v3.bgen", samples_filepath="/lab/corradin_biobank/Raw_UKB_downloads/sample_files/ukb45624_imp_chr9_v3_s487320.sample")

In [22]:
test_bgen_obj = read_bgen("/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr9_v3.bgen", samples_filepath="/lab/corradin_biobank/Raw_UKB_downloads/sample_files/ukb45624_imp_chr9_v3_s487320.sample")

Sample IDs are read from /lab/corradin_biobank/Raw_UKB_downloads/sample_files/ukb45624_imp_chr9_v3_s487320.sample.


Mapping genotypes: 100%|██████████| 4066774/4066774 [08:30<00:00, 7972.36it/s] 


In [31]:
type(test_bgen_obj)

bgen_reader._bgen2.open_bgen

In [56]:
dd.DataFrame

dask.dataframe.core.DataFrame

In [61]:
type(test_bgen_obj["samples"])

pandas.core.series.Series

In [62]:
type(test_bgen_obj["genotype"][0])#["variants"]["id"].compute()

dask.delayed.Delayed

In [33]:
test_bgen_obj["genotype"][0].compute()["probs"]

array([[1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       ...,
       [0.98823529, 0.01176471, 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ]])

In [12]:
test = test_bgen_file_format.load(chrom=9)#.ids

In [43]:
np.where(test.ids in ['9:10163_CT_C', '9:10273_AAC_A', '9:10327_T_C'])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [17]:
test_data_catalog.load("genetic_file_bgen").files.single_file

BgenFileFormat(filepath={1: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr1_v3.bgen'), 2: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr2_v3.bgen'), 3: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr3_v3.bgen'), 4: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr4_v3.bgen'), 5: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr5_v3.bgen'), 6: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr6_v3.bgen'), 7: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr7_v3.bgen'), 8: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr8_v3.bgen'), 9: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr9_v3.bgen'), 10: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr10_v3.bgen'), 11: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr11_v3.bgen'), 12: Path('/lab/corradin_biobank/Raw_UKB_downloads/BGEN/ukb_imp_chr12_v3.bgen'), 13: Path('/lab/corradin_biobank/R